In [ ]:
import torch
import segmentation_models_pytorch as smp
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pathlib
from collections import defaultdict
from typing import List, Dict, Tuple
import umap.umap_ as umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from sklearn.utils import resample
import os
import shutil
import pandas as pd



In [ ]:
symp_value = 76   # Grayscale value for images with pubic symphysis and background mask only
head_value = 150  # Grayscale value for images with head mask and background mask
combined_value = 200  # Arbitrary value for images that have all labels
background_value = 0 #Grayscale value for images with background label only 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set random seeds for reproducibility
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

# Load the final ensemble model
def load_ensemble_model(model_path, device):
    model = smp.Unet(encoder_name="efficientnet-b0", encoder_weights=None, in_channels=1, classes=3)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model = model.to(device)
    model.eval()
    return model

def get_feature_extractor(model):
    # Use the encoder part for feature extraction
    return model.encoder

# Define the preprocessing transform
preprocess = A.Compose([
    A.Normalize(mean=(0.5,), std=(0.5,), max_pixel_value=255.0),
    ToTensorV2()
])

# Function to load and preprocess an image
def load_image(image_path):
    image = Image.open(image_path).convert("L")
    image = np.array(image)
    augmented = preprocess(image=image)
    return augmented['image'].unsqueeze(0)

final_model_path = 'ensemble_model_final.pth'
ensemble_model = load_ensemble_model(final_model_path, device)

# Get the feature extractor
feature_extractor = get_feature_extractor(ensemble_model)

# Function to extract features from the 10th layer
def extract_10th_layer_features(feature_extractor, input_tensor):
    features = []
    x = input_tensor

    # Iterate through the layers of the encoder
    x = feature_extractor._conv_stem(x)
    x = feature_extractor._bn0(x)

    for idx, block in enumerate(feature_extractor._blocks):
        x = block(x)
        if idx == 9:  # 10th layer
            features.append(x)
            break

    return features

# Function to extract features with labels
def extract_features_with_labels(image_paths: List[pathlib.Path], label_paths: List[pathlib.Path], device) -> Dict[int, List[Tuple[pathlib.Path, np.ndarray]]]:
    features = defaultdict(list)
    num_processed = 0
    label_counts = defaultdict(int)

    for img_path, label_path in zip(image_paths, label_paths):
        try:
            img = load_image(img_path).to(device)  # Load and preprocess image
            with torch.no_grad():
                feat = extract_10th_layer_features(feature_extractor, img)
            feat = feat[0].cpu().numpy().squeeze().flatten()  # Extract the feature from the list

            mask = Image.open(label_path).convert('L')
            mask = np.array(mask)
            unique_labels = set(np.unique(mask))

            #print(f"Image {img_path}: Unique labels in mask: {unique_labels}")  # Debugging print

            if unique_labels == {background_value}:
                features[background_value].append((img_path, feat))
                label_counts[background_value] += 1
            else:
                if symp_value in unique_labels and head_value in unique_labels:
                    features[combined_value].append((img_path, feat))
                    label_counts[combined_value] += 1
                else:
                    if symp_value in unique_labels:
                        features[symp_value].append((img_path, feat))
                        label_counts[symp_value] += 1
                    if head_value in unique_labels:
                        features[head_value].append((img_path, feat))
                        label_counts[head_value] += 1

            num_processed += 1
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

    #print(f"Total number of images processed: {num_processed}")
    for label, count in label_counts.items():
        print(f"Label {label}: {count} images")

    return features

# Function to gather all image and label paths
def gather_image_label_paths(image_dir: pathlib.Path, label_dir: pathlib.Path):
    image_paths = list(image_dir.glob('*.png'))
    label_paths = [label_dir / (img_path.stem + '_mask.png') for img_path in image_paths]
    return image_paths, label_paths



In [ ]:
#validation scores on clustering

In [ ]:
support_input_folder = r'C:\Users\cinth\Documentos\ams\data_science\actual_thesis\codes\MedSAM_Universeg_2024\datasets\data\dataset_complete_2\partitioned_dataset_original\images\train'
support_mask_folder = r'C:\Users\cinth\Documentos\ams\data_science\actual_thesis\codes\MedSAM_Universeg_2024\datasets\data\dataset_complete_2\partitioned_dataset_original\masks\train'

image_dir = pathlib.Path(support_input_folder)
label_dir = pathlib.Path(support_mask_folder)


In [ ]:

image_paths, label_paths = gather_image_label_paths(image_dir, label_dir)
features = extract_features_with_labels(image_paths, label_paths, device)

# Iterate over each unique label and perform clustering
for label in features:
    print(f"Processing label {label}...")

    subset_features = np.array([feat for _, feat in features[label]])

    # Perform UMAP to reduce to 2 dimensions
    umap_model = umap.UMAP(n_components=2, random_state=42)
    reduced_features_umap = umap_model.fit_transform(subset_features)

    # Determine the optimal number of clusters using the elbow method and silhouette score
    wcss = []
    silhouette_scores = []
    for i in range(2, 15):  
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=42)
        kmeans.fit(reduced_features_umap)
        wcss.append(kmeans.inertia_)
        cluster_labels = kmeans.predict(reduced_features_umap)
        silhouette_scores.append(silhouette_score(reduced_features_umap, cluster_labels))

    # # Plot WCSS (Elbow Method)
    # plt.plot(range(2, 15), wcss)
    # plt.title(f'Elbow Method for Label {label}')
    # plt.xlabel('Number of clusters')
    # plt.ylabel('WCSS')
    # plt.show()
    # 
    # # Plot Silhouette Scores
    # plt.plot(range(2, 15), silhouette_scores)
    # plt.title(f'Silhouette Score Method for Label {label}')
    # plt.xlabel('Number of clusters')
    # plt.ylabel('Silhouette Score')
    # plt.show()

    # Print WCSS and silhouette scores
    print(f"WCSS for Label {label}: {wcss}")
    print(f"Silhouette Scores for Label {label}: {silhouette_scores}")

    # Choose the optimal number of clusters based on the highest silhouette score
    optimal_k = silhouette_scores.index(max(silhouette_scores)) + 2  # +2 because range starts from 2
    print(f"Optimal number of clusters for Label {label}: {optimal_k}")

    # Apply K-means with the chosen number of clusters
    kmeans = KMeans(n_clusters=optimal_k, random_state=42)
    cluster_labels = kmeans.fit_predict(reduced_features_umap)

    # Visualize the clusters using UMAP
    plt.figure(figsize=(10, 7))
    unique_cluster_labels = np.unique(cluster_labels)
    colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_cluster_labels)))
        
    for cl_label, color in zip(unique_cluster_labels, colors):
        indices = np.where(cluster_labels == cl_label)
        plt.scatter(reduced_features_umap[indices, 0], reduced_features_umap[indices, 1], label=f'Cluster {cl_label}', c=[color])

    # # UMAP Visualization of Clusters for Label
    # plt.title(f'UMAP Visualization of Clusters for Label {label}')
    # plt.xlabel('UMAP Component 1')
    # plt.ylabel('UMAP Component 2')
    # plt.legend()
    # plt.show()

    # Calculate Davies-Bouldin Index
    db_index = davies_bouldin_score(reduced_features_umap, cluster_labels)
    print(f"Davies-Bouldin Index for Label {label}: {db_index}")

    # Calculate Calinski-Harabasz Index
    ch_index = calinski_harabasz_score(reduced_features_umap, cluster_labels)
    print(f"Calinski-Harabasz Index for Label {label}: {ch_index}")



In [ ]:
## Apply clustering and image selection ->select images 20% of the training dataset

In [ ]:
# Define 'num_clusters' and 'num_images' for each label
config = {

    combined_value: {'num_clusters': 14, 'num_images': 33},
    symp_value: {'num_clusters': 4, 'num_images': 10},
    head_value: {'num_clusters': 4, 'num_images': 32},
    background_value: {'num_clusters': 5, 'num_images': 24} 
}

# Apply clustering and image selection based on the configuration
selected_image_paths = []  # List to store selected image paths
image_counts = {}  # Dictionary to store counts of selected images per label

for label, settings in config.items():
    print(f"Processing label {label} with settings: clusters={settings['num_clusters']}, images={settings['num_images']}...")

    subset_features = np.array([feat for _, feat in features[label]])
    umap_model = umap.UMAP(n_components=2, random_state=42)
    reduced_features_umap = umap_model.fit_transform(subset_features)

    # Apply KMeans with the number of clusters 
    kmeans = KMeans(n_clusters=settings['num_clusters'], random_state=42)
    cluster_labels = kmeans.fit_predict(reduced_features_umap)

    count = 0  

    # Find medoids and select closest images
    for cluster_index in range(settings['num_clusters']):
        cluster_points = reduced_features_umap[cluster_labels == cluster_index]
        medoid_index = np.argmin(pairwise_distances(cluster_points, metric='euclidean').sum(axis=1))
        medoid = cluster_points[medoid_index]

        # Calculate distances to medoid and sort
        distances = np.linalg.norm(cluster_points - medoid, axis=1)
        sorted_indices = np.argsort(distances)

        # Select medoid and the number of closest images specified in the config
        indices_to_select = sorted_indices[:settings['num_images'] + 1]  
        for idx in indices_to_select:
            if idx < len(features[label]):
                selected_image_paths.append(features[label][idx][0])
                count += 1

    # Store the count of selected images for the current label
    image_counts[label] = count

# Print the counts of selected images per label
for label, count in image_counts.items():
    print(f"Label {label}: {count} images selected")

# Convert to DataFrame and save to CSV
df = pd.DataFrame(selected_image_paths, columns=['Image_Path'])
df.to_csv('medoids_and_samples_final_30.csv', index=False)
print("Results saved to 'medoids_and_samples_final_20.csv'")



In [ ]:
#Save the representative in one folder

In [ ]:


# Define input and mask folders for the dataset
support_input_folder = r'C:\Users\cinth\Documentos\ams\data_science\actual_thesis\codes\MedSAM_Universeg_2024\datasets\data\dataset_complete\partitioned_dataset_original\images\train'
support_mask_folder = r'C:\Users\cinth\Documentos\ams\data_science\actual_thesis\codes\MedSAM_Universeg_2024\datasets\data\dataset_complete\partitioned_dataset_original\masks\train'

# Path to the CSV file containing the selected image paths
csv_file_path = './medoids_and_samples_final_20.csv'

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Print the content of the DataFrame to debug
print("DataFrame contents:")
print(df)

# Define the source folders for images and masks
source_image_folder = pathlib.Path(support_input_folder)
source_mask_folder = pathlib.Path(support_mask_folder)

# Define the destination folders for selected images and masks
dest_image_folder = pathlib.Path('./ef_selected_images_final_20')
dest_mask_folder = pathlib.Path('./ef_selected_masks_final_20')

# Create destination directories if they don't exist
dest_image_folder.mkdir(parents=True, exist_ok=True)
dest_mask_folder.mkdir(parents=True, exist_ok=True)

# Copy images and their respective masks to the destination folder
total_images_copied = 0
total_masks_copied = 0

for col in df.columns:
    for image_name in df[col].dropna():
        # Ensure the image name is a string path
        if isinstance(image_name, str) or isinstance(image_name, pathlib.PurePath):
            image_path = pathlib.Path(image_name)
            mask_path = source_mask_folder / (image_path.stem + '_mask.png')
            
            # Check if the image and mask exist
            image_exists = image_path.exists()
            mask_exists = mask_path.exists()
            
            print(f"Processing image: {image_path}")
            print(f"Image exists: {image_exists}, Mask exists: {mask_exists}")
            
            if image_exists and mask_exists:
                # Copy the image
                shutil.copy(image_path, dest_image_folder / image_path.name)
                total_images_copied += 1
                # Copy the mask
                shutil.copy(mask_path, dest_mask_folder / mask_path.name)
                total_masks_copied += 1
            else:
                print(f"Error: Image or mask not found for {image_path}")
                if not image_exists:
                    print(f"Image missing: {image_path}")
                if not mask_exists:
                    print(f"Mask missing: {mask_path}")
                # Stop execution
                raise FileNotFoundError(f"Image or mask not found for {image_path}")
        else:
            print(f"Skipping invalid image path: {image_name}")

print(f"Total images copied: {total_images_copied}")
print(f"Total masks copied: {total_masks_copied}")
print(f"Images and masks have been copied to '{dest_image_folder}' and '{dest_mask_folder}' respectively.")
